In [12]:
import parser
import sparse_matrix
import predict

%load_ext autoreload
%autoreload 2

configs = {
    "k": 1.5,
    "b": 0.5,
    "alpha": 1,
    "beta": 0.75,
    "gamma": 0.15,
    "target": 100,
    "rochio": False,
    "query_path": "../queries/query-test.xml",
    "output_path": "../prediction.csv",
    "model_path": "../model",
    "corpus_path": "../CIRB010",

}

if __name__ == '__main__':
#     parser.parse_arg(configs)
    fname_to_id, id_to_fname, id_to_doclen = parser.parse_file_list(configs)
    vocab_to_id, id_to_vocab = parser.parse_vocab_list(configs)
    doc_count = len(fname_to_id)
    inverted_files, gram_to_id, gram_count = parser.parse_inverted_file(configs, doc_count)
    old_inverted_files = inverted_files.copy()
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Reading Inverted Files: 


In [13]:
print(gram_count)
print(doc_count)
configs["gram_count"] = gram_count
configs["doc_count"] = doc_count

1193467
46972


In [14]:
# Save checkpoint for notebook
inverted_files = old_inverted_files.copy()
avdl = sum(id_to_doclen.values()) / len(id_to_doclen)
corpus = {
    "fname_to_id": fname_to_id,
    "id_to_doclen": id_to_doclen,
    "id_to_fname": id_to_fname,
    "vocab_to_id": vocab_to_id,
    "id_to_vocab": id_to_vocab,
    "inverted_files": inverted_files,
    "gram_to_id": gram_to_id,
    "avdl": avdl,
}
corpus["sparse"] = sparse_matrix.gen_matrix(corpus, configs)
# id_to_magnitude = sparse_matrix.gen_id_to_magnitude(corpus, configs)
# corpus["id_to_magnitude"] = id_to_magnitude

0 {'IDF': 9.841037515662922, 'docs': {33689: 1, 38365: 1}, 'max_freq': 1}
1 {'IDF': 10.351863139428913, 'docs': {33256: 1}, 'max_freq': 1}
2 {'IDF': 10.351863139428913, 'docs': {33256: 1}, 'max_freq': 1}
3 {'IDF': 10.351863139428913, 'docs': {10849: 2}, 'max_freq': 2}
4 {'IDF': 10.351863139428913, 'docs': {10849: 1}, 'max_freq': 1}
1193463 {'IDF': 10.351863139428913, 'docs': {32692: 1}, 'max_freq': 1}
1193464 {'IDF': 10.351863139428913, 'docs': {13117: 1}, 'max_freq': 1}
1193465 {'IDF': 10.351863139428913, 'docs': {13117: 1}, 'max_freq': 1}
1193466 {'IDF': 10.351863139428913, 'docs': {30308: 1}, 'max_freq': 1}



  (0, 17145)	4.013536439802942
  (0, 17232)	8.940033559431845
  (0, 17384)	6.909244752577779
  (0, 17595)	2.544576699714985
  (0, 18141)	11.0610461501555
  (0, 20345)	1.5918434473559662
  (0, 20709)	8.45074843390356
  (0, 29889)	3.621771862663303
  (0, 29890)	7.014694363918147
  (0, 34213)	2.4069478445579717
  (0, 34235)	3.5793639237055936
  (0, 43871)	0.6500078556999438
  (0, 44782)	3.367668314300231
  (0, 44998)	0.8427240971451537
  (0, 45041)	3.8956996479929935
  (0, 45565)	0.7541831544489747
  (0, 45615)	2.0492278635141714
  (0, 46407)	0.7794325605020581
  (0, 46483)	4.383871275722084
  (0, 46583)	5.313963965370928
  (0, 46966)	4.983538935012084
  (0, 47006)	0.7493722649053098
  (0, 47444)	3.0810031286043906
  (0, 50894)	0.7588632928750838
  (0, 51120)	1.8143711833793346
  :	:
  (46971, 1142894)	0.8881862939245703
  (46971, 1142991)	2.5585044153166727
  (46971, 1143028)	0.6031712584389829
  (46971, 1143033)	2.647998951833788
  (46971, 1143211)	2.9460978080245623
  (46971, 1143563)	

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [87]:
import parser
import sparse_matrix
import predict
import rocchio
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

configs["title_weight"] = 1
configs["question_weight"] = 1
configs["concepts_weight"] = 1
configs["narrative_weight"] = 1
configs["query_path"] = "../queries/query-train.xml"
configs["cdn"] = 1
configs["ctc"] = 1
configs["cte"] = 1
configs["cts"] = 1
configs["unigram_weight"] = 1
configs["bigram_weight"] = 1
configs["rocchio_iters"] = 1
configs["use_cosine"] = False

print("Processing Query")
queries = parser.parse_queries(corpus, configs, configs["query_path"])
# print(queries[0])
sparse_queries = []
for query in queries:
    sparse_queries.append( sparse_matrix.gen_query_vector(query, corpus, configs) )
# query_to_magnitude = sparse_matrix.gen_query_to_magnitude(queries)
query_responses = []
for sparse_query in tqdm(sparse_queries):
    query_responses.append( predict.predict_query(sparse_query, corpus, configs) )
#     print(response)
for _ in tqdm(range(configs["rocchio_iters"])):
    for i in tqdm(range(len(query_responses))):
        sparse_queries[i] = rocchio.rocchio_feedback(query_responses[i], sparse_queries[i],  corpus, configs)
        response = predict.predict_query(sparse_queries[i], corpus, configs)
        query_responses[i] = response

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing Query


In [88]:
predict.process_predictions(query_responses, configs, corpus)

print(query_responses[0])
predict.write_predictions(query_responses, queries)
predict.calc_MAP(query_responses, configs)

['cts_for_0002013', 'cts_for_0000925', 'cte_foc_0002798', 'cts_for_0001093', 'cdn_soc_0004946', 'cts_soc_0003259', 'cts_for_0002010', 'cts_soc_0003030', 'cte_foc_0000260', 'cdn_soc_0000300', 'cts_for_0000202', 'cts_soc_0002336', 'cdn_loc_0000300', 'cts_soc_0004817', 'cts_soc_0005595', 'cdn_loc_0001911', 'cts_soc_0003580', 'cte_foc_0001543', 'cte_foc_0002784', 'cts_for_0000288', 'cdn_soc_0004947', 'cdn_soc_0002307', 'cdn_soc_0003955', 'cdn_soc_0002385', 'cts_for_0001203', 'cts_for_0000180', 'cte_foc_0003486', 'cts_esy_0000215', 'cdn_ent_0000068', 'cts_ent_0001662', 'cdn_soc_0002219', 'cts_soc_0000801', 'cts_ent_0001002', 'cts_for_0001915', 'cts_for_0000082', 'cts_ent_0003437', 'cts_soc_0006141', 'cts_esy_0000128', 'cte_foc_0001485', 'cts_ent_0000064', 'ctc_dep_0001540', 'cts_foc_0003889', 'cts_soc_0005821', 'cts_foc_0003272', 'cdn_ent_0000749', 'cdn_soc_0002947', 'cdn_soc_0001745', 'cts_ent_0000339', 'cdn_ent_0000029', 'cts_for_0000518', 'cdn_soc_0001627', 'cdn_soc_0004432', 'cdn_foc_00

### TODO!!!
* Add rochio
* Add logarithmic(?)
Add weighting to different components (title, 關鍵字)
Add weighting to different document source (cdn, cts, etc)